In [1]:
import csv
import pandas as pd
import re
import requests
import os
from unidecode import unidecode
from datetime import datetime


# Ingest Data

In [2]:
lineups = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/New Data January 23/Match Lineups _ Goals Final Version - All Lineups.csv')
lineups = lineups.drop(columns={'Unnamed: 0'})

results = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/New Data January 23/Match Results - International Match Results.csv')
results = results.drop(columns={'Unnamed: 0'})

countrycodes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/countries_and_codes.csv')

## Functions

In [3]:
def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countrycodes.loc[countrycodes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

### Add rows to countryCodes database

In [4]:
# Add a new column 'Accents Removed'
countrycodes['Accents Removed'] = countrycodes['Country'].apply(remove_accents)
countrycodes.at[48, 'Country'] = 'China, PR'


data = {'Country': ['British Virgin Islands'], ' Code': ['BVI'], 'Accents Removed': ['British Virgin Islands']}
one_row_dataframe = pd.DataFrame(data)
countrycodes = pd.concat([countrycodes, one_row_dataframe], ignore_index=True)

data = {'Country': ['North Ireland'], ' Code': ['NIR'], 'Accents Removed': ['North Ireland']}
one_row_dataframe = pd.DataFrame(data)
countrycodes = pd.concat([countrycodes, one_row_dataframe], ignore_index=True)




## WORKING on RESULTS

### Finding missing rows & fixing country names in results

#### Team 1

In [5]:
missing = []

for i in range(0, len(results)):
    
    if results.at[i, 'Team 1'] in countrycodes['Country'].values or results.at[i, 'Team 1'] in countrycodes['Accents Removed'].values:
        0==0
    else:
        if(results.at[i, 'Team 1'] in missing):
            0==0
        else:
            missing.append(results.at[i, 'Team 1'])
            print(i, results.at[i, 'Team 1'])


52 nan


#### Team 2

In [6]:
missing2 = []

for i in range(0, len(results)):
    
    if results.at[i, 'Team 2'] in countrycodes['Country'].values or results.at[i, 'Team 2'] in countrycodes['Accents Removed'].values:
        0==0
    else:
        if(results.at[i, 'Team 2'] in missing2):
            0==0
        else:
            missing2.append(results.at[i, 'Team 2'])
            print(i, results.at[i, 'Team 2'])


52 nan


### Adding Country ID to results

In [7]:
results = results.drop(52, axis=0).reset_index().drop(columns='index', axis=1)


In [9]:
def process_date(date):
    # Check if the date is NaN or not a string
    if pd.isna(date) or not isinstance(date, str):
        return date  # Return NaN or non-string values as they are
    elif ' ' in date:
        # Split the date and time
        date_parts = date.split(' ')
        # Keep only the date part
        return date_parts[0]
    else:
        return date

# Apply the function to the 'Date' column
results['Date'] = results['Date'].apply(process_date)


In [10]:
# Add new columns 'Team 1 Code' and 'Team 2 Code'
results['Team 1 Code'] = results['Team 1'].apply(get_country_code)
results['Team 2 Code'] = results['Team 2'].apply(get_country_code)

results = results.reset_index()
results = results.rename(columns={'index': 'Match ID'})


In [11]:
for i in range(0, len(results)):
    if(results.at[i, 'Team 1'] == 'Namibia'):
        results.at[i, 'Team 1 Code'] = 'NA'
    if(results.at[i, 'Team 2'] == 'Namibia'):
        results.at[i, 'Team 2 Code'] = 'NA'

### REsults - handle U-20 games

In [12]:
for i in range(0, len(results)):
    # Check if 'U-20' is present in the 'COMPETITION' column
    if(pd.isna(results.at[i,'Competition'])):
        0==0
    else:
        if 'U-20' in results.at[i,'Competition']:
            # Add 'U20' to the end of 'TEAM 1' and 'TEAM 2'
            results.at[i, 'Team 1'] += ' U-20'
            results.at[i, 'Team 2'] += ' U-20'

            #bit to change
            teams_names = results.at[i, 'SearchString'].split(' on ')[0]

            team_name_1 = teams_names.split(' vs ')[0]
            team_name_2 = teams_names.split(' vs ')[1]

            team_name_1 = team_name_1 + " U-20"
            team_name_2 = team_name_2 + " U-20"

            teams_names = team_name_1 + " vs " + team_name_2

            #bit to keep the same
            rest_of_searchstring = results.at[i, 'SearchString'].split(' on ')[1]

            #putting them together 
            results.at[i, 'SearchString'] = teams_names + " on "+ rest_of_searchstring 


## WORKING ON LINEUPS

### Finding missing rows & fixing country names in Lineups

In [13]:
for i in range(0, len(lineups)):
    if lineups.at[i, 'TeamName'] == 'North\xa0Ireland':
        lineups.at[i, 'TeamName'] = 'North Ireland'

    if(lineups.at[i, 'TeamName'] == 'L\'stein'):
        lineups.at[i, 'TeamName'] = 'Liechtenstein'

    if lineups.at[i, 'TeamName'] == 'San\xa0Marino':
        lineups.at[i, 'TeamName'] = 'San Marino'

    if lineups.at[i, 'TeamName'] == 'Moldova U21':
        lineups.at[i, 'TeamName'] = 'Moldova'

    if lineups.at[i, 'TeamName'] == 'England U21':
        lineups.at[i, 'TeamName'] = 'England'

### Adding country ID to Lineups

#### Senior games

In [14]:
for i in range(0, len(lineups)):
    if(pd.isna(lineups.at[i, 'TeamName'])):
        0==0
    else:
        this_country =  lineups.at[i, 'TeamName']
        if('U-20' in this_country):
            0==0
        else:
            
            code = countrycodes[countrycodes['Country'] == this_country].reset_index().at[0, ' Code']
                #print(i, this_country.split(' U-20')[0], code)
            lineups.at[i, ' Code'] = code
#lineups[' Code'] = lineups['TeamName'].apply(get_country_code)

#### U-20 games

In [15]:
#countrycodes[countrycodes['Country'] == this_country.split(' U-20')[0]].reset_index().at[0, ' Code']

for i in range(0, len(lineups)):
    if(pd.isna(lineups.at[i, 'TeamName'])):
        0==0
    else:
        this_country =  lineups.at[i, 'TeamName']
        if('U-20' in this_country):
            code = countrycodes[countrycodes['Country'] == this_country.split(' U-20')[0]].reset_index().at[0, ' Code']
            #print(i, this_country.split(' U-20')[0], code)
            lineups.at[i, ' Code'] = code

### Ad Hoc Fixing

In [16]:
for i in range(0, len(lineups)):
    if(lineups.at[i, 'TeamName'] == 'Chad'):
        lineups.at[i, ' Code'] = 'TD'
    if(lineups.at[i, 'TeamName'] == 'Namibia'):
        lineups.at[i, ' Code'] = 'NA'
    if(lineups.at[i, 'TeamName'] == 'Curacao'):
        lineups.at[i, ' Code'] = 'CW'

In [17]:
lineups = lineups.dropna(subset=[' Code']).reset_index()


In [18]:
for i in range(0, len(lineups)):
    if(pd.isna(lineups.at[i, ' Code'])):
        print(i, lineups.at[i, 'TeamName'])
        0==0
    else:
        0==0
        #print(i, lineups.at[i, 'TeamName'])

## Adding Match ID to lineups

In [19]:
from datetime import datetime

def format_date(date_str):
    # Parse the input date string
    date_obj = datetime.strptime(date_str, '%d-%b-%y')

    # Check if the day has only one character, add leading zero if true
    day_str = date_obj.strftime('%d').rjust(2, '0')

    # Format the date string with the modified day
    formatted_date_str = date_obj.replace(day=int(day_str)).strftime('%d-%b-%y')

    return formatted_date_str

# Example usage:
input_date = '2-Feb-22'
formatted_date = format_date(input_date)
print(formatted_date)


02-Feb-22


In [20]:
def format_input_date_add_zeroes(input_date):
    # Split the input date into three sections
    sections = input_date.split('-')

    # Add leading zeros if necessary
    sections[0] = sections[0].rjust(2, '0')
    sections[1] = sections[1].rjust(2, '0')

    # Join the sections back together with '-'
    formatted_date = '-'.join(sections)

    return formatted_date

# Example usage:
input_date = '17-19-2021'
formatted_date = format_input_date_add_zeroes(input_date)
print(formatted_date)


17-19-2021


In [21]:
def has_letters(input_str):
    return any(char.isalpha() for char in input_str)

In [22]:
lineups['Found?'] = False
lineups['Match ID'] = 999999

In [ ]:
#results[(results['Date'] == this_row_date)] 
#results[results['Team 2'] == 'Namibia']
#results 
#(lineups.at[3730, 'Date'])

In [40]:
correct_tally = 0
for i in range(0, len(lineups)):
#for i in range(4712, len(lineups)):
#for i in range(0, 4712):

    


#for i in range(5758, 5760):
#for i in range(5932, 5934):



    if(type(lineups.at[i, 'TeamName']) == float):
        0==0
    else:
    
        this_row_team = lineups.at[i, ' Code']
        this_row_comp = lineups.at[i, 'Competition']
        this_row_date = lineups.at[i, 'Date']

        row_match_results_1 = results[(results['Date'] == this_row_date) & (results['Team 1 Code'] == this_row_team)]
        row_match_results_2 = results[(results['Date'] == this_row_date) & (results['Team 2 Code'] == this_row_team)]
        row_match_results = pd.concat([row_match_results_1, row_match_results_2], ignore_index=True)

        if(len(row_match_results) == 1):
            correct_tally += 1
            correct_match_ID = row_match_results.at[0, 'Match ID']
            lineups.at[i, 'Match ID'] = correct_match_ID
        elif(len(row_match_results) >= 2):
            print('more than 1 results', i, this_row_team, lineups.at[i, 'Match'], lineups.at[i, 'Competition'], this_row_date)
        elif(len(row_match_results) == 0):

            if(has_letters(this_row_date)):
                formatted_date = format_date(this_row_date)

                row_match_results_1 = results[(results['Date'] == formatted_date) & (results['Team 1 Code'] == this_row_team)]
                row_match_results_2 = results[(results['Date'] == formatted_date) & (results['Team 2 Code'] == this_row_team)]
                new_df_datechange = pd.concat([row_match_results_1, row_match_results_2], ignore_index=True)

                if(len(new_df_datechange) == 1):
                    correct_tally += 1
                    correct_match_ID = new_df_datechange.at[0, 'Match ID']
                    lineups.at[i, 'Match ID'] = correct_match_ID
                else:
                    print("no rows month was a name",i, this_row_team, lineups.at[i, 'Match'], lineups.at[i, 'Competition'], formatted_date)
            
            else:
                formatted_date_plus_zeroes = format_input_date_add_zeroes(this_row_date)

                row_match_results_1 = results[(results['Date'] == formatted_date_plus_zeroes) & (results['Team 1 Code'] == this_row_team)]
                row_match_results_2 = results[(results['Date'] == formatted_date_plus_zeroes) & (results['Team 2 Code'] == this_row_team)]
                new_df_datechange = pd.concat([row_match_results_1, row_match_results_2], ignore_index=True)

                if(len(new_df_datechange) == 1):
                    correct_tally += 1
                    correct_match_ID = new_df_datechange.at[0, 'Match ID']
                    lineups.at[i, 'Match ID'] = correct_match_ID
                else:
                    print("no rows added zeroes",i, this_row_team, lineups.at[i, 'Match'], lineups.at[i, 'Competition'], formatted_date_plus_zeroes)



#match ID column 



more than 1 results 110 IT Ukraine U-20 vs Italy U-20 U-20 World Cup 2019 11.06.19
more than 1 results 114 UA Colombia U-20 vs Ukraine U-20 U-20 World Cup 2019 07.06.19
more than 1 results 1295 UA Ukraine vs Serbia Euros 2020, Qualifiers 07.06.19
more than 1 results 1705 IT Italy vs Bosnia Euros 2020, Qualifiers 11.06.19
more than 1 results 5586 AF Afghanistan vs Syria World Cup Qualifiers, Asia, 2018 11-06-2015
more than 1 results 5587 SY Afghanistan vs Syria World Cup Qualifiers, Asia, 2018 11-06-2015
no rows added zeroes 5758 SA Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
no rows added zeroes 5759 PS Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
more than 1 results 5856 SA Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
more than 1 results 5857 PS Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
no rows added zeroes 5932 AF Afghanistan vs Syria World Cup Qualifiers, Asia, 2018 11-06-2015
no 

In [95]:
rows_to_drop = [5856, 5857, 5932, 5933, 7150, 7151]

lineups = lineups.drop(rows_to_drop, axis=0)

In [97]:
lineups[(lineups['Match ID'] == 999999)]

,index,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
209,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
314,314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
443,443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
572,572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
701,701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
1222,1222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
1747,1747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
2206,2206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
2309,2309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999


In [93]:
lineups.at[110, 'Match ID'] = 54
lineups.at[114, 'Match ID'] = 56
lineups.at[1295, 'Match ID'] = 650
lineups.at[1705, 'Match ID'] = 855
lineups.at[5586, 'Match ID'] = 2810
lineups.at[5587, 'Match ID'] = 2810


lineups.at[5758, 'Match ID'] = 2896
lineups.at[5759, 'Match ID'] = 2896

lineups.at[6968, 'Match ID'] = 3503
lineups.at[6969, 'Match ID'] = 3503


In [98]:
#rows to delete in results
##row 2983 Afghanistan vs Syria 
##row 2945 Saudi Arabia vs Palestina
##row 3594 Israel vs North Macedonia
rows_to_drop_results = [2983, 2945, 3594]

results = results.drop(rows_to_drop_results, axis=0)


In [113]:
lineups.at[6100, 'TeamName'] = "Iran"
lineups.at[6100, ' Code'] = "IR"

lineups.at[6100, 'Match ID'] = 3067

lineups.at[6099, 'Match'] = "Cambodia vs Iran"
lineups.at[6100, 'Match'] = "Cambodia vs Iran"



In [117]:
lineups[lineups['Match ID'] == 999999]

,index,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID


In [115]:
lineups = lineups.dropna(subset=['TeamName'])


In [118]:
lineups

,index,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
0,0,Brazil U-20,-,-,-,-,-,-,-,-,...,-,-,-,-,"U-20 World Cup, 2015",20.06.15,Brazil U-20 vs Serbia U-20,BR,False,0
1,1,Serbia U-20,-,-,-,-,-,-,-,-,...,-,-,-,-,"U-20 World Cup, 2015",20.06.15,Brazil U-20 vs Serbia U-20,RS,False,0
2,2,Senegal U-20,-,-,-,-,-,-,-,-,...,-,-,-,-,"U-20 World Cup, 2015",20.06.15,Senegal U-20 vs Mali U-20,SN,False,1
3,3,Mali U-20,-,-,-,-,-,-,-,-,...,-,-,-,-,"U-20 World Cup, 2015",20.06.15,Senegal U-20 vs Mali U-20,ML,False,1
4,4,Brazil U-20,-,-,-,-,-,-,-,-,...,-,-,-,-,"U-20 World Cup, 2015",17.06.15,Brazil U-20 vs Senegal U-20,BR,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8596,8596,Qatar,22 Meshaal Barsham,13 Musaab Khidir,2 Ahmed Suhail,5 Tarek Salman,14 Homam Ahmed,12 Abdullah Marafee,10 Mohammed Waad,6 Ahmed Fathi,...,-,-,-,-,"Gold Cup, 2023",03-07-2023,Mexico vs Qatar,QA,False,4321
8597,8597,Guatemala,1 N. Hagen,7 A. Herrera,4 J. Pinto,3 N. Samayoa,2 J. Ardón,17 O. Castellanos,8 R. Saravia,18 N. Méndez-Laing,...,-,-,-,-,"Gold Cup, 2023",02-07-2023,Guatemala vs Canada,GT,False,4322
8598,8598,Canada,18 M. Borjan,14 M. Lumpungu,5 S. Vitória,4 K. Miller,22 R. Laryea,20 A. Ahmed,3 L. Fraser,21 J. Osorio,...,-,-,-,-,"Gold Cup, 2023",02-07-2023,Guatemala vs Canada,CA,False,4322
8599,8599,El Salvador,1 M. González,21 B. Tamacas,4 E. Zavaleta,2 E. Cabalceta,15 Á. Roldán,6 N. Orellana,17 J. Henriquez,8 B. Landaverde,...,-,-,-,-,"Gold Cup, 2023",01-07-2023,El Salvador vs Costa Rica,SV,False,4323


# EXPORTING DATA

In [120]:
results.to_csv('newest_results_data.csv', index=False)


In [119]:
lineups.to_csv('newest_lineups_data.csv', index=False)
